# Test Article

Загружаем библиотеки

In [10]:
from transformers import AutoTokenizer, AutoModel, pipeline

In [52]:
modelpath = "../model/fine-train"
# modelpath = "DeepPavlov/rubert-base-cased"
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained(modelpath)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [53]:
text = "После начала российской военной спецоперации на Украине западные страны усилили санкционное давление на Москву. МИД России призвал стороны прекратить боевые действия."

In [56]:
import re

tr = 0.001

def mask_bert_sent(text, model):
    unmasker = pipeline("fill-mask", model=model)
    p = re.compile(r'[\w-]+')
    iter = p.finditer(text)
    listErr = []
    for match in iter:
        if match.start() == 0:
            masktext = "[MASK]" + text[match.end():]
        elif match.end() == len(text):
            masktext = text[:match.start()] + "[MASK]"
        else:
            masktext = text[:match.start()] + "[MASK]" + text[match.end():]
        
        res = unmasker(masktext, targets=[match.group()])
        if res[0]['score'] < tr:
            errorrDesc = {
                "word": match.group(),
                "start": match.start(),
                "end": match.end()
            }
            listErr.append(errorrDesc)
    return listErr


In [57]:
errors = mask_bert_sent(text, modelpath)
if len(errors) > 0:
    for e in errors:
        print (e)

The specified target token `санкционное` does not exist in the model vocabulary. Replacing with `санкцион`.


{'word': 'спецоперации', 'start': 32, 'end': 44}
{'word': 'санкционное', 'start': 80, 'end': 91}
